# Imports and setting up viz

NB : conda env1 on PC, lam1env on spirit (Python3.12)

In [ ]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

#import personnal tools
import sys
sys.path.append('../../python_tools/')
from tools import *

In [13]:
# rivers = cfeature.NaturalEarthFeature('physical', 'rivers_lake_centerlines', '10m',edgecolor=(0, 0, 0, 0.3), facecolor='none')

# Load and edit files

LAM output analysis.

simu irr et simu no-irr.

## Area and period selection

In [14]:
lon_min=-19
lon_max=12
lat_min=28
lat_max=53

lon_min=-10
lon_max=4
lat_min=34
lat_max=46


date_min = '2010-01-01'
date_max = '2022-12-31'

In [15]:
#filter to keep only certain period
# season='SON'
# sim = sim.where(sim['time.season'] == season, drop=True)
# simirr = simirr.where(simirr['time.season'] == season, drop=True)

## Sims

In [16]:
# noirr_dir='../../../JZ_simu_outputs/LAM/LAM_1000_40/noirr_2010_2022'
# irr_dir='../../../JZ_simu_outputs/LAM/LAM_1000_40/irr_2010_2022'
noirr_dir='../../../JZ_simu_outputs/LAM/LAM_1500_60/noirr'
irr_dir='../../../JZ_simu_outputs/LAM/LAM_1500_60/irr'

In [ ]:
#sim
filename = '{}/*/ATM/TS_MO/*.nc'.format(noirr_dir)
# filename = '{}/*/ATM/MO/*.nc'.format(noirr_dir)

sim0 = xr.open_mfdataset(filename)
sim0.attrs['name'] = 'no_irr'
sim = sim0.rename({'time_counter':'time'})
sim = sim.sel(lon=slice(lon_min, lon_max),lat=slice(lat_min, lat_max))

sim['evap'] = sim['evap'] *3600 * 24
sim['evap'].attrs['units'] = 'mm/d'
sim['evap'].attrs['long_name'] = 'Evapotranspiration'

sim['precip'] = sim['precip'] *3600 * 24
sim['precip'].attrs['units'] = 'mm/d'
sim['precip'].attrs['long_name'] = 'Total precipitation'

sim['fluxsens']= -sim['sens']

sim['netrad'] = sim['LWdnSFC'] - sim['LWupSFC'] + sim['SWdnSFC'] - sim['SWupSFC']
sim['netrad'].attrs['units'] = 'W/m2'

sim['SWnetSFC'] = sim['SWdnSFC'] - sim['SWupSFC']
sim['SWnetSFC'].attrs['units'] = 'W/m2'

sim['LWnetSFC'] = sim['LWdnSFC'] - sim['LWupSFC']
sim['LWnetSFC'].attrs['units'] = 'W/m2'

sim['P - E'] = sim['precip'] - sim['evap']
sim['P - E'].attrs['units'] = 'mm/d'

# sim['Altitude'] = sim['phis'] / 9.81
# sim['Altitude'].attrs['units'] = 'm'

sim

In [ ]:
#sim irr
filename = '{}/*/ATM/TS_MO/*.nc'.format(irr_dir)
# filename = '{}/*/ATM/MO/*.nc'.format(irr_dir)

sim0irr = xr.open_mfdataset(filename)
sim0irr.attrs['name'] = 'irr'
simirr = sim0irr.rename({'time_counter':'time'})
simirr = simirr.sel(lon=slice(lon_min, lon_max),lat=slice(lat_min, lat_max))

simirr['evap'] = simirr['evap'] *3600 * 24
simirr['evap'].attrs['units'] = 'mm/d'

simirr['precip'] = simirr['precip'] *3600 * 24
simirr['precip'].attrs['units'] = 'mm/d'

simirr['fluxsens']= -simirr['sens']

simirr['netrad'] = simirr['LWdnSFC'] - simirr['LWupSFC'] + simirr['SWdnSFC'] - simirr['SWupSFC']
simirr['netrad'].attrs['units'] = 'W/m2'

simirr['SWnetSFC'] = simirr['SWdnSFC'] - simirr['SWupSFC']
simirr['SWnetSFC'].attrs['units'] = 'W/m2'

simirr['LWnetSFC'] = simirr['LWdnSFC'] - simirr['LWupSFC']
simirr['LWnetSFC'].attrs['units'] = 'W/m2'

simirr['P - E'] = simirr['precip'] - simirr['evap']
simirr['P - E'].attrs['units'] = 'mm/d'

# simirr['Altitude'] = simirr['phis'] / 9.81
# simirr['Altitude'].attrs['units'] = 'm'

simirr


## Obs

### ERA5

In [ ]:
#open ERA5 file
# filename = '../../../obs/ERA5_eval_2010_2022.nc'
filename = '../../../obs/ERA5_large_eval.nc'

era = xr.open_mfdataset(filename)
era.attrs['name'] = 'ERA5'
#define or edit vars
#name ERA5 vars like in LMDZOR outputs
dict =  {
        'longitude':'lon',
        'latitude':'lat',
        'e':'evap',
        'ro':'totrunoff',
        'tp':'precip',
        'u10':'u10m',
        'v10':'v10m',
        'p71.162' : 'uq',
        'p72.162' : 'vq',
        'sshf' : 'fluxsens',
        'slhf' : 'fluxlat',
        'si10' : 'sfcWind',
        'ssrd' : 'SWdnSFC',
        'strd' : 'LWdnSFC',
        'ssr' : 'SWnetSFC',
        'str' : 'LWnetSFC',
        }
era = era.rename(dict)
era=era.sel(time=slice(date_min, date_max))
era = era.sel(lon=slice(lon_min,lon_max),lat=slice(lat_max, lat_min))

#make evap positive in era
era['evap'] = -era['evap'] * 1000
era['evap'].attrs['units'] = 'mm/d'
era['precip'] = era['precip']*1000
era['precip'].attrs['units'] = 'mm/d'
era['fluxsens'] = -era['fluxsens'] / (3600 *24)
era['fluxsens'].attrs['units'] = 'W/m2'
# era['cldt'] = era['tcc']*100
# era['cldt'].attrs['units'] = '%'

era['P - E'] = era['precip'] - era['evap']
era['P - E'].attrs['units'] = 'mm/d'

era['calcWind'] = np.sqrt(era['u10m']**2 + era['v10m']**2)
era['calcWind'].attrs['units'] = 'm/s'
   
#convert 4 radiation variables from J/m2 to W/m2
era['SWdnSFC'] = era['SWdnSFC'] / (3600 *24)
era['SWdnSFC'].attrs['units'] = 'W/m2'
era['LWdnSFC'] = era['LWdnSFC'] / (3600 *24)
era['LWdnSFC'].attrs['units'] = 'W/m2'
era['SWnetSFC'] = era['SWnetSFC'] / (3600 *24)
era['SWnetSFC'].attrs['units'] = 'W/m2'
era['LWnetSFC'] = era['LWnetSFC'] / (3600 *24)
era['LWnetSFC'].attrs['units'] = 'W/m2'
#add up vars and netrad
era['LWupSFC'] = - era['LWnetSFC'] + era['LWdnSFC']
era['LWupSFC'].attrs['units'] = 'W/m2'
era['SWupSFC'] = - era['SWnetSFC'] + era['SWdnSFC']
era['SWupSFC'].attrs['units'] = 'W/m2'
era['netrad'] = era['LWnetSFC'] + era['SWnetSFC']
era['netrad'].attrs['units'] = 'W/m2'

era


In [38]:
# #open tqruv file
# filename='../../../obs/TQRUV_850_2010_2022_monthly.nc'
# tqruv = xr.open_dataset(filename)
# tqruv.attrs['name'] = 'ERA5'
# tqruv = tqruv.sel(longitude=slice(-13,6),latitude=slice(49,32))
# #rename longitude and latitude to lon and lat
# tqruv = tqruv.rename({'longitude':'lon','latitude':'lat'})

# #rename all variables to add 850 in name
# tqruv['u850'] = tqruv['u']
# tqruv['v850'] = tqruv['v']
# tqruv['q850'] = tqruv['q']
# tqruv['t850'] = tqruv['t']
# tqruv['r850'] = tqruv['r']
# tqruv

### ERA-Land

In [ ]:
filename = '../../../obs/ERA5-Land.nc'
eraland = xr.open_dataset(filename)
eraland.attrs['name'] = 'ERA5-Land'
rename_dict = {'longitude':'lon',
               'latitude':'lat',
               'valid_time':'time',
               'skt':'temp_sol',
               'sro':'runoff',
               'ro':'mrro',
               'ssr':'SW_net',
               'str':'LW_net',
               'sshf':'fluxsens',
               'slhf':'fluxlat',
               'tp':'precip',
               'src':'fastr'
               }
eraland = eraland.rename(rename_dict)

eraland['mrsos'] = eraland['swvl1']*0.1*1000
eraland['mrsos'].attrs['units'] = 'mm'
#average variables swl1 to swvl4
eraland['lvl1-4_ave'] = (eraland['swvl1'] * 7 + eraland['swvl2'] * 21 + eraland['swvl3'] * 72 + eraland['swvl4'] * 189 )  / 289
eraland['humtot'] = eraland['lvl1-4_ave']*2*1000
eraland['humtot'].attrs['units'] = 'mm'

eraland['evap'] = - eraland['fluxlat'] * 0.408 * 0.0864
eraland['evap'].attrs['units'] = 'mm/d'

eraland

### GLEAM

In [ ]:
#gleam
# filename='../../../obs/evap/E_1980-2022_GLEAM_v3.8a_MO.nc'
# gleam0=xr.open_dataset(filename)
# gleam0.attrs['name']='GLEAM'
# gleam = gleam0.sel(lon=slice(lon_min, lon_max),lat=slice(lat_max, lat_min))
# #adapt period
# gleam=gleam.sel(time=slice('2010-01-01', '2022-12-31'))
# #name gleam vars like in LMDZOR outputs
# dict =  {
#         'E':'evap'
#         }
# gleam = gleam.rename(dict)
# #convert mm/month to mm/day
# gleam['evap']=gleam['evap']/31
# gleam['evap'].attrs['units']='mm/d'
# gleam

filename = '../../../obs/GLEAMv4.1a/E/*'
gleam = xr.open_mfdataset(filename)
gleam.attrs['name'] = 'GLEAMv4.1a'
#restrict lon lat
gleam = gleam.sel(lon=slice(lon_min,lon_max),lat=slice(lat_max,lat_min))
#rename var SM
gleam['evap']=gleam['E'] / 31
gleam['evap'].attrs['units'] = 'mm/d'
gleam

### FluxCom

In [ ]:
filename='../../..//obs/FluxCom/ensemble_rs-_meteo_ALL/monthly/*.RS_METEO.EBC-*.MLM-ALL.METEO-ALL.720_360.monthly.*.nc'
fluxcom0=xr.open_mfdataset(filename)
fluxcom=fluxcom0.sel(lon=slice(-13,6.25),lat=slice(lat_max, lat_min))
fluxcom=fluxcom.sel(time=slice('2010-01-01', '2022-12-31'))

fluxcom.attrs["name"]='FluxCom'

fluxcom['evap'] = fluxcom['LE'] * 0.408 # To convert from MJ/m2/d to mm/d
fluxcom['evap'].attrs['units'] = 'mm/d'

fluxcom['fluxlat'] = fluxcom['LE'] / 0.0864 # To convert from MJ/m2/d to W/m2
fluxcom['fluxlat'].attrs['units'] = 'W/m²'

fluxcom['fluxsens'] = fluxcom['H'] / 0.0864 # To convert from MJ/m2/d to W/m2
fluxcom['fluxsens'].attrs['units'] = 'W/m²'

fluxcom['netrad'] = fluxcom['Rn'] / 0.0864 # To convert from MJ/m2/d to W/m2
fluxcom['netrad'].attrs['units'] = 'W/m²'
fluxcom

### GPCC

In [ ]:
#gpcc
filename='../../../obs/precips/precip.mon.total.0.25x0.25.v2020.nc'
gpcc0=xr.open_mfdataset(filename)
gpcc0.attrs['name'] = 'GPCC'
gpcc=gpcc0.sel(time=slice('2010-01-01', '2022-12-31'))
gpcc['lon'] = ((gpcc['lon'] + 180) % 360) - 180
gpcc1 = gpcc.sortby('lon')

# gpcc=gpcc.sel(lon=slice(-13,6.25),lat=slice(lat_min, lat_max))
gpcc=gpcc1.sel(lon=slice(-13,6.25),lat=slice(lat_max, lat_min))

# gpcc1=gpcc.sel(lon=slice(347,360),lat=slice(lat_max, lat_min))
# gpcc2=gpcc.sel(lon=slice(0,6.25),lat=slice(lat_max, lat_min))
# gpcc=xr.combine_by_coords([gpcc1, gpcc2])
# gpcc = gpcc.where(gpcc['lon'] < 6.25, drop=True)

# gpcc['lon']=gpcc['lon']-180.0
# gpcc=gpcc.sel(lon=slice(-13,6.25))

gpcc['precip'] = gpcc['precip'] / 30 #convert to mm/d
gpcc['precip'].attrs['units'] = 'mm/d'
gpcc

## Masking, interpolation, and time period selection

### Sims and products interp to products

In [25]:
# sim_iera = sim.interp_like(era)
# sim_igpcc = sim.interp_like(gpcc)
# sim_igleam = sim.interp_like(gleam)
# sim_ifluxcom = sim.interp_like(fluxcom)
# # sim_itqruv = sim.interp_like(tqruv)

# simirr_iera = simirr.interp_like(era)
# simirr_igpcc = simirr.interp_like(gpcc)
# simirr_igleam = simirr.interp_like(gleam)
# simirr_ifluxcom = simirr.interp_like(fluxcom)
# # simirr_itqruv = simirr.interp_like(tqruv)

In [26]:
# #same masks but adatped when interpolated on obs grid
# con_mask_iera=sim_iera['contfracOR']>0.9
# ip_mask_iera=polygon_to_mask(sim_iera, iberian_peninsula)

# con_mask_igpcc=sim_igpcc['contfracOR']>0.9
# ip_mask_igpcc=polygon_to_mask(sim_igpcc, iberian_peninsula)

# con_mask_igleam=sim_igleam['contfracOR']>0.9
# ip_mask_igleam=polygon_to_mask(sim_igleam, iberian_peninsula)

# con_mask_ifluxcom=sim_ifluxcom['contfracOR']>0.9
# ip_mask_ifluxcom=polygon_to_mask(sim_ifluxcom, iberian_peninsula)

In [27]:
# ip_sim_iera = ip_sim.interp_like(era)
# ip_sim_igpcc = ip_sim.interp_like(gpcc)
# ip_sim_igleam = ip_sim.interp_like(gleam)
# ip_sim_ifluxcom = ip_sim.interp_like(fluxcom)
# ip_sim_itqruv = ip_sim.interp_like(tqruv)

# ip_simirr_iera = ip_simirr.interp_like(era)
# ip_simirr_igpcc = ip_simirr.interp_like(gpcc)
# ip_simirr_igleam = ip_simirr.interp_like(gleam)
# ip_simirr_ifluxcom = ip_simirr.interp_like(fluxcom)
# ip_simirr_itqruv = ip_simirr.interp_like(tqruv)

In [28]:
# sim_iipera = sim.interp_like(era).where(con_mask_iera).where(ip_mask_iera['mask'], drop=False)
# sim_iipgpcc = sim.interp_like(ip_gpcc).where(con_mask_igpcc).where(ip_mask_igpcc['mask'], drop=False)
# sim_iipgleam = sim.interp_like(ip_gleam).where(con_mask_igleam).where(ip_mask_igleam['mask'], drop=False)
# sim_iipfluxcom = sim.interp_like(ip_fluxcom).where(con_mask_ifluxcom).where(ip_mask_ifluxcom['mask'], drop=False)

# simirr_iipera = simirr.interp_like(era).where(con_mask_iera).where(ip_mask_iera['mask'], drop=False)
# simirr_iipgpcc = simirr.interp_like(ip_gpcc).where(con_mask_igpcc).where(ip_mask_igpcc['mask'], drop=False)
# simirr_iipgleam = simirr.interp_like(ip_gleam).where(con_mask_igleam).where(ip_mask_igleam['mask'], drop=False)
# simirr_iipfluxcom = simirr.interp_like(ip_fluxcom).where(con_mask_ifluxcom).where(ip_mask_ifluxcom['mask'], drop=False)

### Products interp to sim

In [ ]:
#continental fraction mask (LMDZ outputs)
con_mask=sim['contfracOR']>0.9
ip_mask=polygon_to_mask(sim, iberian_peninsula)

In [30]:
cont_sim=sim.where(con_mask)
cont_simirr=simirr.where(con_mask)

ip_sim = sim.where(ip_mask['mask'], drop=False).where(con_mask)
ip_simirr = simirr.where(ip_mask['mask'], drop=False).where(con_mask)

In [31]:
# era5
# cont_era=era.where(con_mask_iera)
# ip_era = cont_era.where(ip_mask_iera['mask'], drop=False)

era_isim=era.interp_like(sim)
cont_era_isim=era_isim.where(con_mask)
ip_era_isim = cont_era_isim.where(ip_mask['mask'], drop=False)

In [32]:
#era-land
# cont_eraland=eraland.where(con_mask)
# ip_eraland = eraland.where(ip_mask['mask'], drop=False)

eraland_isim=eraland.interp_like(sim)
cont_eraland_isim=eraland_isim.where(con_mask)
ip_eraland_isim = cont_eraland_isim.where(ip_mask['mask'], drop=False)

In [33]:
#gleam
# cont_gleam=gleam.where(con_mask_igleam)
# ip_gleam = cont_gleam.where(ip_mask_igleam['mask'], drop=False)

gleam_isim=gleam.interp_like(sim)
cont_gleam_isim=gleam_isim.where(con_mask)
ip_gleam_isim = cont_gleam_isim.where(ip_mask['mask'], drop=False)

# gleam_iipera=gleam.interp_like(era).where(con_mask_iera).where(ip_mask_iera['mask'], drop=False)

In [34]:
#fluxcom
# cont_fluxcom=fluxcom.where(con_mask_ifluxcom)
# ip_fluxcom = cont_fluxcom.where(ip_mask_ifluxcom['mask'], drop=False)

fluxcom_isim=fluxcom.interp_like(sim)
cont_fluxcom_isim=fluxcom_isim.where(con_mask)
ip_fluxcom_isim = cont_fluxcom_isim.where(ip_mask['mask'], drop=False)

# fluxcom_iipera=fluxcom.interp_like(era).where(con_mask_iera).where(ip_mask_iera['mask'], drop=False)

In [35]:
#gpcc
# cont_gpcc=gpcc.where(con_mask_igpcc)
# ip_gpcc = cont_gpcc.where(ip_mask_igpcc['mask'], drop=False)

gpcc_isim=gpcc.interp_like(sim)
cont_gpcc_isim=gpcc_isim.where(con_mask)
ip_gpcc_isim = cont_gpcc_isim.where(ip_mask['mask'], drop=False)

# gpcc_iipera=gpcc.interp_like(era).where(con_mask_iera).where(ip_mask_iera['mask'], drop=False)

## Diff datasets

In [79]:
sim_diff=simirr-sim
ip_sim_diff=ip_simirr-ip_sim

In [80]:
#era
sim_era_diff=sim - era_isim
ip_sim_era_diff=ip_simirr - ip_era_isim
#mean of both diff datasets
sim_era_diff_mean = sim_era_diff.mean(dim='time')
ip_sim_era_diff_mean = ip_sim_era_diff.mean(dim='time')

In [81]:
#gleam
sim_gleam_diff=sim - gleam_isim
ip_sim_gleam_diff=ip_simirr - ip_gleam_isim
#mean of both diff datasets
sim_gleam_diff_mean = sim_gleam_diff.mean(dim='time')
ip_sim_gleam_diff_mean = ip_sim_gleam_diff.mean(dim='time')

In [82]:
#gpcc
sim_gpcc_diff=sim - gpcc_isim
ip_sim_gpcc_diff=ip_simirr - ip_gpcc_isim
#mean of both diff datasets
sim_gpcc_diff_mean = sim_gpcc_diff.mean(dim='time')
ip_sim_gpcc_diff_mean = ip_sim_gpcc_diff.mean(dim='time')

# Maps

In [ ]:
#Display map of var for ds
var='evap'
ds=sim
color_map=emb_neutral

vmin=-0.0
vmax=6.5
# vmin=None
vmin,vmax=None, None

map_ave(ds, var, cmap=color_map, vmin=vmin, vmax=vmax, poly=None)

In [ ]:
#Diff map
var='evap'
ds1=simirr
# ds2=ip_era_isim
# ds2=ip_fluxcom_isim
ds2=sim
# ds2=ip_gpcc_isim

max_value= 1
min_value=-1
# max_value=None
# min_value=None
min_value,max_value=None, None

map_diff_ave(ds1, ds2, var, vmin=min_value, vmax=max_value, cmap=emb_neutral, hex=False, sig=False)
# diff=ds1[var]-ds2[var]
# map_seasons(diff, cmap=emb_neutral, vmin=min_value, vmax=max_value, title=title, hex=False

In [ ]:
#map for 4 seasons
var='evap'
ds1=ip_sim
ds2=ip_gleam_isim
max_value= 1
min_value=-1

diff=ds1[var]-ds2[var]
title='{} bias, {} vs {} ({})'.format( var, ds1.attrs['name'], ds2.attrs['name'], ds1[var].attrs['units'])
plotvar=diff

# plotvar=ds1[var]
# title='P-E (mm/d, {})'.format(ds1.attrs['name'])

map_seasons(plotvar, cmap=emb_neutral, vmin=min_value, vmax=max_value, title=title, hex=False)

In [ ]:
# Display 2 maps of var
var = 'SWupSFC'
min_value=None
max_value=None
color_map=wet
ds1=sim_iera
ds2=era
map_two_ds(ds1, ds2, var, vmin=min_value, figsize=(15,6), vmax=max_value, cmap=color_map)

In [ ]:
#Display map of var for ds on restricted area
var='evap'
ds = sim
latmin=36.7
latmax=36.9
lonmin=-6.4
lonmax=-6.2

#restrict ds to latmin, latmax, lonmin, lonmax
ds = ds.where(ds.lat >= latmin, drop=True).where(ds.lat <= latmax, drop=True)
ds = ds.where(ds.lon >= lonmin, drop=True).where(ds.lon <= lonmax, drop=True)
plotvar = ds[var].mean(dim='time')
map_plotvar(plotvar, cmap=emb_neutral_neutral, vmax=None, title=var)

In [ ]:
#Relative diff map
var='precip'
max_value=None
ds1=cont_fluxcom
ds2=ip_fluxcom
# plt.title('Gross primary production relative difference (%)')
map_rel_diff_ave(ds1, ds2, var, vmax=max_value)

# Time series

In [ ]:
var='precip'

ds1=ip_sim
ds2=ip_simirr
ds3=ip_era_isim
ds4=ip_gleam_isim
# ds5=ip_fluxcom_isim
ds5=ip_eraland_isim
ds4=ip_gpcc_isim

ds_list=[ds1, ds2, ds3, ds4]#, ds5]
# ds_list=[fluxcom]

year_min=2010
year_max=2019
time_series_ave(ds_list, var, year_min=year_min, year_max=year_max, title='{} {}'.format(var, ds1[var].attrs['units']))
seasonal_cycle_ave(ds_list, var, year_min=year_min, year_max=year_max, title='Seasonal cycle of {} ({}, {}-{})'.format(var, ds1[var].attrs['units'], year_min, year_max))

# Scatter plots

In [ ]:
var1='precip'
ds1=ip_sim_era_diff_mean
#remove values where precip is smaller than 5
mask=ds1[var1] < 2
ds1=ds1.where(mask)

var2='evap'
# ds2=ip_sim_era_diff
ds2=ds1

scatter_vars(ds1, ds2, var1, var2, reg=False)

In [ ]:
var1='precip'
ds1=ip_sim_gpcc_diff_mean
#remove values where precip is smaller than 2
mask=ds1[var1] < 2
ds1=ds1.where(mask)

var2='evap'
ds2=ip_sim_gleam_diff_mean
ds2=ds2.where(mask)
scatter_vars(ds1, ds2, var1, var2, reg=False)